In [2]:
from rasterio.plot import show
from rasterio.merge import merge
import rasterio as rio
from rasterio import features
import rioxarray

from pathlib import Path
import glob
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
from osgeo import gdal
import os

In [3]:
path = r'C:/Uni Jena/GEO 411/Data/Sentinel-2/NDWI_20m_arbeitsgebiete/'
Path(path + 'Thueringen').mkdir(parents=True, exist_ok=True)
output_path = path + 'Thueringen'
Path(path + 'Unterfranken').mkdir(parents=True, exist_ok=True)
output_path_unt = path + 'Unterfranken'

path_ndvi = r'C:/Uni Jena/GEO 411/Data/Sentinel-2/NDVI_20m_arbeitsgebiete/'
Path(path_ndvi + 'Thueringen').mkdir(parents=True, exist_ok=True)
output_path_ndvi = path_ndvi + 'Thueringen'
Path(path_ndvi + 'Unterfranken').mkdir(parents=True, exist_ok=True)
output_path_unt_ndvi = path_ndvi + 'Unterfranken'

In [4]:
def get_all_paths(path, ort):
    filepath_list1 = [i.split('\\')[1] for i in glob.glob(path + ort+"_1/" +"*.tiff")]
    filepath_list2 = [i.split('\\')[1] for i in glob.glob(path + ort+"_2/" +"*.tiff")]
    filepath_list3 = [i.split('\\')[1] for i in glob.glob(path + ort+"_3/" +"*.tiff")]
    filepath_list4 = [i.split('\\')[1] for i in glob.glob(path + ort+"_4/" +"*.tiff")]
    filepath_concat = filepath_list1+filepath_list2+filepath_list3+filepath_list4
    return filepath_concat

In [5]:
filepath_concat = get_all_paths(path=path, ort="Thueringen")
filepath_concat_u = get_all_paths(path=path, ort="Unterfranken")
filepath_concat_ndvi = get_all_paths(path=path_ndvi, ort="Thueringen")
filepath_concat_u_ndvi = get_all_paths(path=path_ndvi, ort="Unterfranken")

In [6]:
def get_unique_filepaths(filepath_concat):
    filepath_list = list()

    for item in filepath_concat:
        if item not in filepath_list:
            filepath_list.append(item)
    return filepath_list

In [7]:
filepath_list = get_unique_filepaths(filepath_concat=filepath_concat)
filepath_list_u = get_unique_filepaths(filepath_concat=filepath_concat_u)
filepath_list_ndvi = get_unique_filepaths(filepath_concat=filepath_concat_ndvi)
filepath_list_u_ndvi = get_unique_filepaths(filepath_concat=filepath_concat_u_ndvi)

In [8]:
def get_merge_list(filepath_list, path, ort):
    liste = []
    for i in range(len(filepath_list)):
        liste_da = []
        if os.path.isfile(path + ort+"_1/" + filepath_list[i]):
            da1 = path + ort+"_1/" + filepath_list[i]
            liste_da.append(da1)
        if os.path.isfile(path + ort+"_2/" + filepath_list[i]):
            da2 = path + ort+"_2/" + filepath_list[i]
            liste_da.append(da2)
        if os.path.isfile(path + ort+"_3/" + filepath_list[i]):
            da3 = path + ort+"_3/" + filepath_list[i]
            liste_da.append(da3)
        if os.path.isfile(path + ort+"_4/" + filepath_list[i]):
            da4 = path + ort+"_4/" + filepath_list[i]
            liste_da.append(da4)
        liste.append(liste_da)
    return liste

In [9]:
liste_da_all = get_merge_list(filepath_list=filepath_list, path=path, ort="Thueringen")
liste_da_all_u = get_merge_list(filepath_list=filepath_list_u, path=path, ort="Unterfranken")
liste_da_all_ndvi = get_merge_list(filepath_list=filepath_list_ndvi, path=path_ndvi, ort="Thueringen")
liste_da_all_u_ndvi = get_merge_list(filepath_list=filepath_list_u_ndvi, path=path_ndvi, ort="Unterfranken")

In [10]:
def merge_tiff(liste_da_all, output_path):
    for i in range(len(liste_da_all)):
        g = gdal.Warp(output_path+"/"+liste_da_all[i][0][-18:], liste_da_all[i], format="GTiff",
                          options=["COMPRESS=LZW", "TILED=YES"]) # if you want
        g = None # Close file and flush to disk

In [11]:
#NDWI Thueringen 20m
merge_tiff(liste_da_all=liste_da_all, output_path=output_path)

In [187]:
#NDWI Unterfranken 20m
merge_tiff(liste_da_all=liste_da_all_u, output_path=output_path_unt)

In [188]:
#NDVI Thueringen 20m
merge_tiff(liste_da_all=liste_da_all_ndvi, output_path=output_path_ndvi)

In [189]:
#NDVI Unterfranken 20m
merge_tiff(liste_da_all=liste_da_all_u_ndvi, output_path=output_path_unt_ndvi)